In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import sys
from scipy.stats import gaussian_kde
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib.ticker import MaxNLocator
import pathlib
import warnings
warnings.filterwarnings(
    "ignore",
    category=pd.errors.PerformanceWarning,
    message="DataFrame is highly fragmented.*"
)
plt.rcParams.update({'font.size': 22})

In [ ]:
if 'win' in sys.platform:
    cospath = "E:/OneDrive/PhD/PhD/Data/Hintereisferner/Output/aws_eb/"
    awspath = "E:/OneDrive/PhD/PhD/Data/Hintereisferner/Output/point_ensemble_lhs/best/"
    aws_path = "E:/OneDrive/PhD/PhD/Data/Hintereisferner/Climate/AWS_Obleitner/"
else:
    cospath = "/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Output/aws_eb/"
    awspath = "/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Output/point_ensemble_lhs/best/"
    aws_path = "/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Climate/AWS_Obleitner/"
   
    
aws_upper = pd.read_csv(aws_path+"Fix_HEFupper_01102003_24102004.csv", parse_dates=True, index_col="time")
aws_upper = aws_upper.loc["2003-10-01":"2004-09-30"]
aws_upper['T'] = aws_upper['T']+273.15
aws_upper['sfc'] = aws_upper['sfc'] - aws_upper['sfc'][0]

aws_upper_copy = aws_upper.copy()

vars_seb_fluxes=['SWnet','LWnet','H','LE','B','QRR','ME']
vars_moisture = ["PRES","RH2","T2","U2","REFREEZE","CONDENSATION","SUBLIMATION","EVAPORATION","DEPOSITION"]
#mass fluxes next?

In [ ]:
# Create AWS reference
aws_upper['SWnet'] = aws_upper['SWI'] - aws_upper['SWO']
aws_upper['LWnet'] = aws_upper['LWI'] - aws_upper['LWO']
aws_upper_hourly = aws_upper.copy()

aws_upper = aws_upper.resample("1D").agg(
    T=('T', lambda x: x.mean() if x.notna().all() else np.nan),
    Dir=('Dir', lambda x: x.mean() if x.notna().all() else np.nan),
    U=('U', lambda x: x.mean() if x.notna().all() else np.nan),
    SWIsum=('SWI', lambda x: x.sum(min_count=len(x)) if x.notna().all() else np.nan),
    SWImean=('SWI', lambda x: x.mean() if x.notna().all() else np.nan),
    SWOsum=('SWO', lambda x: x.sum(min_count=len(x)) if x.notna().all() else np.nan),
    SWOmean=('SWO', lambda x: x.mean() if x.notna().all() else np.nan),
    LWnet_mean=('LWnet', lambda x: x.mean() if x.notna().all() else np.nan),
    SWnet_mean=('SWnet', lambda x: x.mean() if x.notna().all() else np.nan),
    LWO=('LWO', lambda x: x.mean() if x.notna().all() else np.nan),
    LWI=('LWI', lambda x: x.mean() if x.notna().all() else np.nan),
    sfc=('sfc', lambda x: x.mean() if x.notna().all() else np.nan),
    RH=('RH', lambda x: x.mean() if x.notna().all() else np.nan),
    P=('P', lambda x: x.mean() if x.notna().all() else np.nan),
)
aws_upper['alpha'] = aws_upper['SWOsum'] / aws_upper['SWIsum']
aws_upper['sfc'] = aws_upper['sfc'] - aws_upper['sfc'][0]
aws_upper

In [ ]:
date_index = pd.date_range(start='2003-10-01T00:00:00', end='2004-09-30T23:00:00', freq='H')
# Create an empty DataFrame with that datetime index
empty_df = pd.DataFrame(index=date_index)

## prepare empty dictionaries
dic_cos = {}
dic_aws = {}

for var in vars_seb_fluxes + ['G','ALBEDO', 'LWin', 'LWout'] + vars_moisture:
    dic_cos[var] = empty_df.copy()
    dic_aws[var] = empty_df.copy()

In [ ]:
for fp in pathlib.Path(awspath).glob('*.nc'):
    print(fp)
    ds = xr.open_dataset(fp)
    num = str(fp.stem).split('num')[-1]
    ds = ds.sel(time=slice("2003-10-01T00:00:00", None))
    ds = ds.isel(lat=0,lon=0)
    ds['SWnet'] = ds['G'] * (1 - ds['ALBEDO'])
    ds['LWnet'] = ds['LWin'] + ds['LWout']
    ds = ds[vars_seb_fluxes + ['G', 'ALBEDO','LWin','LWout'] + vars_moisture] 
    #key = str(fp.stem).split('RRR-')[-1]
    for var in vars_seb_fluxes + ['G','ALBEDO','LWin','LWout'] + vars_moisture:
        sub = ds[var].to_dataframe().drop(['lat','lon'], axis=1)
        if var == 'ME':
            dic_aws[var][num] = sub[var]*-1
        else:
            dic_aws[var][num] = sub[var]

In [ ]:
for fp in pathlib.Path(cospath).glob('*.csv'):
    print(fp)
    df = pd.read_csv(fp, parse_dates=True, index_col="time")
    df = df.loc["2003-10-01T00:00:00":"2004-09-30T23:00:00"]
    df['LWnet'] = df['LWin'] + df['LWout']
    #EB terms
    df = df[vars_seb_fluxes + ['G', 'ALBEDO','LWin','LWout'] + vars_moisture] 
    key = str(fp.stem).split('RRR-')[-1]
    #dfs = []
    for var in vars_seb_fluxes + ['G','ALBEDO','LWin','LWout'] +  vars_moisture:
        if var == "ME":
            dic_cos[var][key] = df[var]*-1
        else:
            dic_cos[var][key] = df[var]

In [ ]:
med_hourly = dic_cos["G"].median(axis=1)
ci_hourly = dic_cos["G"].quantile(0.975, axis=1)

diff = med_hourly - ci_hourly
diff.plot()

In [ ]:
med_hourly = dic_cos["ALBEDO"].median(axis=1)
ci_hourly = dic_cos["ALBEDO"].quantile(0.975, axis=1)

diff = med_hourly - ci_hourly
diff.plot()

In [ ]:
med_hourly = dic_cos["SWnet"].median(axis=1)
ci_hourly = dic_cos["SWnet"].quantile(0.975, axis=1)

diff = med_hourly - ci_hourly
diff.plot()

In [10]:
## Load filtered dataframe
if 'win' in sys.platform:
    filtered_data = pd.read_csv("E:/OneDrive/PhD/PhD/Data/Hintereisferner/COSIPY/point_scale/pointLHS-tenbestrmse-filtered-params.csv")
else:
    filtered_data = pd.read_csv("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/COSIPY/point_scale/pointLHS-tenbestrmse-filtered-params.csv")

# Calculate weights
"""
log_likelihoods = filtered_data['joint_like'].values
max_ll = np.max(log_likelihoods)
unnormalized_weights = np.exp(log_likelihoods - max_ll)
weights = unnormalized_weights / np.sum(unnormalized_weights)
filtered_data['weight'] = weights
print("Weights calculated.")
filtered_data_indexed = filtered_data.set_index('num')
"""
joint_rmse_scores = filtered_data['joint_rmse'].values
max_rmse_in_ensemble = np.max(joint_rmse_scores)
unnormalized_weights = max_rmse_in_ensemble - joint_rmse_scores
weights = unnormalized_weights / np.sum(unnormalized_weights)
filtered_data['weight'] = weights
filtered_data_indexed = filtered_data.set_index('num')


In [ ]:
filtered_data

In [12]:
def weighted_quantile(data, weights, quantiles):
    """Calculates weighted quantiles of a 1D array."""
    data = np.asarray(data)
    sort_indices = np.argsort(data)
    sorted_data = data[sort_indices]
    sorted_weights = weights[sort_indices]
    cumulative_weights = np.cumsum(sorted_weights)
    cumulative_weights /= cumulative_weights[-1]
    indices = np.searchsorted(cumulative_weights, quantiles)
    indices = np.clip(indices, 0, len(sorted_data) - 1)
    return sorted_data[indices]

In [ ]:
weight_map = {}
column_order = dic_aws['SWnet'].columns # Get the exact column order

for col_name in column_order:
    # Extract the run number string by splitting the name
    #run_num_str = col_name.split('_num')[-1]
    run_num = int(col_name)
    
    # Look up the pre-calculated weight using the run number
    weight = filtered_data_indexed.loc[run_num, 'weight']
    weight_map[col_name] = weight
    
weights_series = pd.Series(weight_map).loc[column_order]
weights = weights_series.values # Final weights array
print("Weights prepared and mapped to data columns.")
weights_series

In [ ]:
eb_results_raw = {}
quantiles = [0.025, 0.50, 0.975]

for component, sdf in dic_aws.items():
    print(f"Processing {component}...")
    
    # Apply the weighted_quantile function to each row (each hour)
    stats_df = sdf.apply(
        lambda row: weighted_quantile(row, weights=weights, quantiles=quantiles),
        axis=1, result_type='expand'
    )
    stats_df.columns = ['lower_ci_95', 'median', 'upper_ci_95']
    eb_results_raw[component] = stats_df


In [ ]:
eb_results_raw['SWnet']

In [16]:
## First stats then rolling mean, otherwise we dilute high likelihood day with low likelihood day
eb_results_smoothed = {}
#rolling_window_hours = 14 * 24

aws_obs_lwnet = aws_upper_hourly['LWnet'].rolling(window="5D", center=True).mean()
aws_obs_swnet = aws_upper_hourly['SWnet'].rolling(window="5D", center=True).mean()

for component, sdf in eb_results_raw.items():
    #smoothed_df = sdf.rolling(window="14D", center=True).mean()
    smoothed_df = sdf.rolling(window="5D", center=True).mean()
    eb_results_smoothed[component] = smoothed_df

In [ ]:
## Order needs to swtich

#resample to daily means
for var in vars_seb_fluxes+['LWout','LWin','G','ALBEDO']+vars_moisture:
    print(var)
    #dic_aws[var] = dic_aws[var].resample("1M").mean()
    #dic_aws[var] = eb_results_smoothed[var]
    #dic_cos[var] = dic_cos[var].resample("1M").mean()
    try:
        dic_cos[var].drop(['median','ci_lower','ci_upper'], axis=1, inplace=True)
    except:
        print("Running for first time.")
        pass
    dic_cos[var]['median'] = dic_cos[var].median(axis=1)
    dic_cos[var]['ci_lower'] = dic_cos[var].quantile(0.025, axis=1)
    dic_cos[var]['ci_upper'] = dic_cos[var].quantile(0.975, axis=1)
    

In [18]:
import copy
dic_aws_raw = copy.deepcopy(eb_results_raw)
dic_cos_raw = copy.deepcopy(dic_cos)

#for component, sdf in dic_aws_raw.items():
#    dic_aws_raw[component] = sdf.resample("1D").mean()
#    dic_cos_raw[component] = dic_cos_raw[component].resample("1D").mean()

In [19]:
## Do the smoothing now - rolling mean?
cosmo_eb_results_smoothed = {}
#rolling_window_hours = 14 * 24

for component, sdf in dic_cos.items():
    #smoothed_df = sdf[['median','ci_lower','ci_upper']].rolling(window="14D", center=True).mean()
    smoothed_df = sdf[['median','ci_lower','ci_upper']].rolling(window="5D", center=True).mean()
    cosmo_eb_results_smoothed[component] = smoothed_df

In [20]:
dic_aws_medians = {}
dic_cos_medians = {}
#
dic_aws_cilower = {}
dic_cos_cilower = {}
#
dic_aws_ciupper = {}
dic_cos_ciupper = {}
###
for var in vars_seb_fluxes + ['LWout','LWin','G']:
    dic_aws_medians[var] = eb_results_smoothed[var]["median"]
    dic_cos_medians[var] = cosmo_eb_results_smoothed[var]['median']
    #
    dic_aws_cilower[var] = eb_results_smoothed[var]["lower_ci_95"]
    dic_cos_cilower[var] = cosmo_eb_results_smoothed[var]['ci_lower']
    #
    dic_aws_ciupper[var] = eb_results_smoothed[var]["upper_ci_95"]
    dic_cos_ciupper[var] = cosmo_eb_results_smoothed[var]['ci_upper']


In [21]:
color_dic = {
    'SWnet': '#1f78b4',  # strong blue
    'LWnet': '#ff7f00',  # deep orange
    'H':     '#33a02c',  # rich green
    'LE':    '#e31a1c',  # clear red
    'B':     '#a65628',  # dark brown
    'QRR':   '#6a3d9a',  # dark purple
    'ME':    '#fb9a99'   # soft salmon-pink (less saturated, still distinct)
}


In [ ]:
fig, axes = plt.subplots(4,2, figsize=(20,12), dpi=150)
ax = axes.flatten()
for i, param in enumerate(['parRRR_factor', 'paralb_ice', 'paralb_snow', 'paralb_firn',	'paralbedo_aging', 'paralbedo_depth', 'parroughness_ice']):
    ax[i].hist(filtered_data[param], edgecolor="black", bins=10)
    ax[i].set_xlabel(param)
    
fig.tight_layout()


In [23]:
## Load AWS and modelled albedo time series - create a time running degree of goodness of fit and "shade" things when this goodness is bad to highlight periods of comparability
med_aws_alb = dic_aws_raw['ALBEDO'][['median']].resample("1D").mean().rename(columns={"median":"AWS"})
med_cos_alb = dic_cos_raw['ALBEDO'][['median']].resample("1D").mean().rename(columns={"median":"COS"})

#
merged_alb = aws_upper[['alpha']].join([med_aws_alb, med_cos_alb], how='inner')
#
merged_alb['MAE_AWS'] = np.abs(merged_alb['alpha'] - merged_alb['AWS'])
merged_alb['MAE_COS'] = np.abs(merged_alb['alpha'] - merged_alb['COS'])

In [ ]:
# do check that they have same index
fig, ax = plt.subplots(2,1, figsize=(16,9), dpi=300)
ax[0].plot(merged_alb.index, merged_alb['alpha'], label="Obs.", color="black")
ax[0].plot(merged_alb.index, merged_alb['AWS'], label="AWS")
ax[0].plot(merged_alb.index, merged_alb['COS'], label="COS")
ax[0].legend()
#
ax[1].plot(merged_alb.index, merged_alb['MAE_AWS'], label="Obs-AWS")
ax[1].plot(merged_alb.index, merged_alb['MAE_COS'], label="Obs-COS")
ax[1].axhline(y=0.2, linestyle="dashed")

ax[1].legend()

In [ ]:
merged_alb.index = pd.to_datetime(merged_alb.index, errors='coerce')
print(merged_alb.index.isna().sum())

In [26]:
# match the rolling means timestamps
mae_hourly = merged_alb[['MAE_AWS','MAE_COS']].reindex(dic_aws_medians['G'].index, method='ffill')

def get_highlight_ranges(mae_series, threshold=0.2):
    """Return list of (start, end) tuples where mae_series >= threshold."""
    highlight = mae_series >= threshold
    ranges = []
    start = None

    for idx, val in highlight.items():
        if val and start is None:
            start = idx
        elif not val and start is not None:
            end = prev_idx
            ranges.append((start, end))
            start = None
        prev_idx = idx

    # Catch last period if it ends at the last row
    if start is not None:
        ranges.append((start, prev_idx))

    return ranges

# Assuming mae_hourly is already aligned to rolling mean timestamps
ranges_AWS = get_highlight_ranges(mae_hourly['MAE_AWS'], threshold=0.2)
ranges_COS = get_highlight_ranges(mae_hourly['MAE_COS'], threshold=0.2)


In [27]:
## Load observed LWnet and SWnet


In [ ]:
fig, ax = plt.subplots(3,1, figsize=(14,12), dpi=300, sharex=True)
for var in vars_seb_fluxes:
    ax[0].plot(dic_cos_medians[var], label=var, color=color_dic[var], zorder=6)
    ax[0].fill_between(dic_cos_medians[var].index, y1=dic_cos_cilower[var], y2=dic_cos_ciupper[var], color=color_dic[var], alpha=0.3, zorder=3)
    ax[0].axhline(y=0, color="black", lw=0.7, zorder=1)
    ax[0].grid()
    #
    ax[1].plot(dic_aws_medians[var], label=var, color=color_dic[var], zorder=6)
    ax[1].fill_between(dic_aws_medians[var].index, y1=dic_aws_cilower[var], y2=dic_aws_ciupper[var], color=color_dic[var], alpha=0.3, zorder=3)
    ax[1].axhline(y=0, color="black", lw=0.7, zorder=1)
    ax[1].grid()
    #
    median_diff = dic_cos_medians[var] - dic_aws_medians[var]
    ax[2].plot(median_diff, label=var, color=color_dic[var], zorder=6)
    ax[2].axhline(y=0, color="black", lw=0.7, zorder=1)
    ax[2].grid()
    
ax[0].plot(aws_obs_lwnet.index, aws_obs_lwnet, label="Obs. LWnet", color="black", linestyle="dotted", zorder=6)
ax[0].plot(aws_obs_swnet.index, aws_obs_swnet, label="Obs. SWnet", color="black", linestyle="dashed", zorder=6)
#measured LWnet and SWnet to AWS plot
ax[1].plot(aws_obs_lwnet.index, aws_obs_lwnet, label="Obs. LWnet", color="black", linestyle="dotted", zorder=6)
ax[1].plot(aws_obs_swnet.index, aws_obs_swnet, label="Obs. SWnet", color="black", linestyle="dashed", zorder=6)

handles, labels = ax[1].get_legend_handles_labels()

# Add figure-level legend below all axes
dic_labels = {'SWnet': r"$Q_{SWnet}$",
              'LWnet': r"$Q_{LWnet}$",
              'H': r"$Q_{H}$",
              'LE': r"$Q_{E}$",
              'B': r"$Q_{G}$",
              'ME': r"$Q_{M}$",
              'QRR': r'$Q_{R}$',
              'Obs. LWnet': r'$Obs_{LWnet}$',
              'Obs. SWnet': r'$Obs_{SWnet}$'}
new_labels = [dic_labels[x] for x in labels]
fig.legend(handles, new_labels, loc='lower center', ncol=5, bbox_to_anchor=(0.5, -0.07))

# Optional: add titles or labels
#ax[0].set_title("Posterior ensemble")
#ax[1].set_title("AWS-forced Ensemble")
#ax[2].set_title("COSMO - AWS Differences")
ax[0].set_ylabel("Fluxes (Wm$^{-2}$)")
ax[1].set_ylabel("Fluxes (Wm$^{-2}$)")
ax[2].set_ylabel("Flux Difference (Wm$^{-2}$)")

#
ax[0].text(0.02, 0.98, 'a) Posterior ensemble', transform=ax[0].transAxes, fontsize=24, va='top')
ax[1].text(0.02, 0.98, 'b) AWS-assisted ensemble', transform=ax[1].transAxes, fontsize=24, va='top')
ax[2].text(0.02, 0.98, r'c) $\Delta$ (Posterior - AWS-assisted ensemble)', transform=ax[2].transAxes, fontsize=24, va='top')

ax[0].set_yticks(np.arange(-220, 220+55,55))
ax[0].set_ylim(-220, 220)
ax[1].set_yticks(np.arange(-220, 220+55,55))
ax[1].set_ylim(-220, 220)
ax[2].set_yticks(np.arange(-120, 120+20,20))
ax[2].set_ylim(-120, 120)
#
for a in [ax[0], ax[1], ax[2]]:
    ticks = a.get_yticks()
    labels = [f"{tick:.0f}" if i % 2 == 0 else "" for i, tick in enumerate(ticks)]
    a.set_yticklabels(labels)
#
ax[2].xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 2, 3,4,5,6,7,8,9,10,11,12)))
days = mdates.DayLocator(interval=7)
ax[2].xaxis.set_minor_locator(days)
ax[2].xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(ax[2].xaxis.get_major_locator()))
ax[2].set_xlim(dic_cos_medians[var].index[0], dic_cos_medians[var].index[-1])

ax[2].xaxis.set_tick_params(labelsize=20, rotation=30)
#ax[2].set_xlabel('Time', fontsize=22)

# Define hydrological seasons
winter_months = [10, 11, 12, 1, 2, 3]  # Oct–Mar
summer_months = [4, 5, 6, 7, 8, 9]     # Apr–Sep

# Create seasonal shading
start_date = dic_cos_medians[var].index[0]
end_date = dic_cos_medians[var].index[-1]
current = start_date

while current < end_date:
    season = "winter" if current.month in winter_months else "summer"
    
    # Determine season end (either end of Mar or Sep)
    if season == "winter":
        # If starting in Jan-Mar, go to end of Mar; if Oct-Dec, go to end of Mar next year
        year = current.year if current.month < 10 else current.year + 1
        season_end = pd.Timestamp(f"{year}-03-31")
    else:
        season_end = pd.Timestamp(f"{current.year}-09-30")
    
    # Clip to data range
    season_end = min(season_end, end_date)
    
    # Define color
    color = "#c6dbef" if season == "winter" else "#fee0d2"  # light blue and light red
    
    # Add shaded region
    for a in ax:
        a.axvspan(current, season_end, color=color, alpha=0.2, zorder=0)
    
    # Move to next season
    current = season_end + pd.Timedelta(days=1)

# add quality hatching
def add_hatching(ax_list, ranges, color, hatch, zorder=2):
    for start, end in ranges:
        for a in ax_list:
            a.axvspan(start, end, facecolor='none', hatch=hatch,
                      edgecolor=color, alpha=0.1, zorder=zorder)
            
aws_color = '#333333'   # dark grey
cos_color = '#1b9e77'   # teal
aws_hatch = '////'
cos_hatch = '\\\\\\\\'

add_hatching([ax[0]], ranges_COS, cos_color, cos_hatch, zorder=2)
add_hatching([ax[1]], ranges_AWS, aws_color, aws_hatch, zorder=2)
    
add_hatching([ax[2]], ranges_AWS, aws_color, aws_hatch, zorder=2)
add_hatching([ax[2]], ranges_COS, cos_color, cos_hatch, zorder=2) 
    
fig.tight_layout()

"""
if 'win' in sys.platform:
    #plt.savefig("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/seb_14dayrollingmean_aws-cosipy.png", bbox_inches="tight")
    print("skip")
    #plt.savefig("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/fig09_seb_5dayrollingmean_aws-cosipy.pdf", bbox_inches="tight")
else:
    #plt.savefig("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/seb_14dayrollingmean_aws-cosipy.png", bbox_inches="tight")
    plt.savefig("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/Fig09_seb_5dayrollingmean_aws-cosipy.png", bbox_inches="tight")

"""


In [ ]:
## Print out raw values over time, print out RWnet, do simple plots here for paper section
dic_aws.keys()


In [ ]:
rnet_aws = dic_aws_medians['LWnet'] + dic_aws_medians['SWnet']
rnet_cos = dic_cos_medians['LWnet'] + dic_cos_medians['SWnet']

rnet_bias = rnet_cos - rnet_aws 

rnet_bias.plot(figsize=(16,9))


In [ ]:
#dic_cos_medians['H'].loc["2004-04-30":"2004-09-30"].max()
#dic_cos_medians['H'].loc["2003-10-01":"2004-04-30"].max()
(dic_cos_medians['ME'].loc["2004-05-01":"2004-09-30"] - dic_aws_medians['ME'].loc["2004-05-01":"2004-09-30"]).min()

In [ ]:
#dic_cos_medians['LWnet'].loc["2003-10-01":"2004-09-30"].mean()
dic_cos_medians['H'].loc["2004-07-01":"2004-08-01"].mean()
#dic_cos_medians['SWnet'].loc["2004-04-30":"2004-09-30"].max()
#dic_cos_medians['G'].loc["2003-10-01":"2004-04-30"].max()
#dic_cos_medians['SWnet'].loc["2004-04-30":"2004-09-30"].min()
#dic_cos_medians['G'].loc["2003-10-01":"2004-04-30"].min()

In [33]:
dt = 3600
rho_water = 1000
lat_heat_melting = 334000

# Calculate melt depth for each hour
dic_cos_medians['hourly_melt_mwe'] = dic_cos_medians['ME'] * dt / (rho_water * lat_heat_melting)
dic_aws_medians['hourly_melt_mwe'] = dic_aws_medians['ME'] * dt / (rho_water * lat_heat_melting)



In [ ]:
dic_cos_medians['hourly_melt_mwe'].loc["2003-10-01":"2004-09-30"].sum() - dic_aws_medians['hourly_melt_mwe'].loc["2003-10-01":"2004-09-30"].sum()

In [ ]:
dic_cos_medians['hourly_melt_mwe']["2004-06-01":"2004-07-15"].sum() - dic_aws_medians['hourly_melt_mwe']["2004-06-01":"2004-07-15"].sum()

In [ ]:
dic_aws_medians['ME'].loc["2004-05-01":"2004-09-30"].sum()

In [ ]:
(dic_cos_medians['ME'].loc["2004-06-10":"2004-07-14"] - dic_aws_medians['ME'].loc["2004-06-10":"2004-07-14"]).max()

In [ ]:
rnet_bias.loc[(mae_hourly['MAE_AWS'] < 0.2) & (mae_hourly['MAE_COS'] < 0.2)].max()


In [ ]:
##some stats
# mod - aws -- mod = 10, aws=5  = 5, so positive == mod overestimats, negative underestimates
print(rnet_bias.mean())
print(rnet_bias.loc["2003-10-01":"2004-04-30"].mean())
print(rnet_bias.loc["2004-04-30":"2004-09-30"].mean())
print("\n-------------")
#
print(rnet_bias.min())
print(rnet_bias.loc["2003-10-01":"2004-04-30"].min())
print(rnet_bias.loc["2004-04-30":"2004-09-30"].min())
print("\n-------------")
#
print(rnet_bias.max())
print(rnet_bias.loc["2003-10-01":"2004-04-30"].max())
print(rnet_bias.loc["2004-04-30":"2004-09-30"].max())

In [40]:
## printing of data for use in text

for var in ["REFREEZE","CONDENSATION","EVAPORATION","SUBLIMATION","DEPOSITION"]:
    dic_aws_raw[var] = dic_aws_raw[var] * 1000 #mm w.e
    dic_cos_raw[var] = dic_cos_raw[var] * 1000
    


In [41]:
## Load plotting script to compare fluxes in more detail
unit_dic = {'SWnet': r"Wm$^{-2}$",
            'LWnet': r"Wm$^{-2}$",
            'H': r"Wm$^{-2}$",
            'LE': r"Wm$^{-2}$",
            'B': r"Wm$^{-2}$",
            'ME': r"Wm$^{-2}$",
            'G': r"Wm$^{-2}$",
            'LWin': r"Wm$^{-2}$",
            'Q2': r"gkg$^{-1}$",
            'RH2': "%",
            'PRES': "hPa",
            'T2': "K",
            'U2': r"ms$^{-1}$",
            'REFREEZE': "mm w.e.",
            'CONDENSATION': "mm w.e.",
            'EVAPORATION': "mm w.e.",
            'SUBLIMATION': "mm w.e.",
            'DEPOSITION': "mm w.e.",
            'Td': "°C"}

ylim_dic = {'SWnet': (-1,625),
            'LWnet': (-250, 75),
            'H':(-100, 150),
            'LE': (-150, 125),
            'B': (-75, 120),
            'ME': (-550, 5),
            'G': (-1, 1300),
            'LWin': (75, 400),
            'Q2': (-0.1, 12),
            'RH2': (-1, 100),
            'PRES': (670, 725),
            'T2': (245, 290),
            'Td': (-40, 15),
            'U2': (-0.1, 16),
            'DEPOSITION': (-0.001, 0.06),
            'CONDENSATION': (-0.001,0.18),
            'REFREEZE': (-0.001, 2.6),
            'SUBLIMATION': (-0.18,0.001)}

In [42]:
def calc_cdf(vals):
    x = np.sort(vals)
    y = np.arange(1, len(x)+1)/len(x)
    
    return x,y

def sc_stats(expected_vals, predicted_vals, drop_nan=False):
    if drop_nan:
        exp_nans = np.isnan(expected_vals)
        expected_vals = expected_vals[~exp_nans]
        predicted_vals = predicted_vals[~np.isnan(predicted_vals)]
        predicted_vals = predicted_vals[~exp_nans]
        expected_vals = expected_vals.loc[expected_vals.index.isin(predicted_vals.index)]
        predicted_vals = predicted_vals.loc[predicted_vals.index.isin(expected_vals.index)]
        expected_vals = expected_vals.loc[expected_vals.index.isin(predicted_vals.index)]
    xy = np.vstack([expected_vals, predicted_vals])
    z = gaussian_kde(xy)(xy)
    #sort points by density, so that densest points are plotted last
    idx = z.argsort()
    expected, predicted, z = expected_vals[idx], predicted_vals[idx], z[idx]
    return expected, predicted, z

In [43]:
def plot_ts(axes, var):
    if var in ["G","LWin","U2","T2","RH2","Td","Q2","PRES"]:
        if var in ["G","LWin"]:
            axes.plot(dic_aws_raw[var]['median'], color="black", label="AWS", zorder=5)
            axes.plot(dic_cos_raw[var]['median'], color="red", label="COSMO", zorder=6)
        else:
            axes.plot(dic_aws_raw[var]['median'], color="black", label="AWS", zorder=6)
            axes.plot(dic_cos_raw[var]['median'], color="red", label="COSMO", zorder=5)
        axes.fill_between(dic_aws_raw[var]['median'].index, dic_cos_raw[var]['ci_lower'],
                        dic_cos_raw[var]['ci_upper'], color="brown",
                    alpha=0.5, label = "")
        axes.fill_between(dic_aws_raw[var].index, dic_aws_raw[var]['lower_ci_95'],
                        dic_aws_raw[var]['upper_ci_95'], color="grey",
                alpha=0.5, label = "")
    else:
        if var in ["SWnet","LWnet"]:
            axes.plot(dic_aws_raw[var]['median'], color="black", label="AWS-assisted Ens. Median", zorder=5)
            axes.plot(dic_cos_raw[var]['median'], color="red", label="Posterior Ens. Median", zorder=6)
        else:
            axes.plot(dic_aws_raw[var]['median'], color="black", label="AWS-assisted Ens. Median", zorder=6)
            axes.plot(dic_cos_raw[var]['median'], color="red", label="Posterior Ens. Median", zorder=5)
        axes.fill_between(dic_aws_raw[var]['median'].index, dic_cos_raw[var]['ci_lower'],
                        dic_cos_raw[var]['ci_upper'], color="brown",
                    alpha=0.5, label = "Posterior 95% CI")
        axes.fill_between(dic_aws_raw[var].index, dic_aws_raw[var]['lower_ci_95'],
                        dic_aws_raw[var]['upper_ci_95'], color="grey",
                alpha=0.5, label = "AWS-assisted 95% CI")
    ## daily rolling mean
    """
    if var in ["G","LWin","U2","T2","RH2","Td","Q2","PRES"]:
        axes.plot(dic_aws_raw[var]['median'].rolling("1D", center=True).mean(), color="black", label="AWS", zorder=6)
        axes.plot(dic_cos_raw[var]['median'].rolling("1D", center=True).mean(), color="red", label="COSMO", zorder=5)
        axes.fill_between(dic_aws_raw[var]['median'].rolling("1D", center=True).mean().index, dic_cos_raw[var]['ci_lower'].rolling("1D", center=True).mean(),
                        dic_cos_raw[var]['ci_upper'].rolling("1D", center=True).mean(), color="brown",
                    alpha=0.5, label = "")
        axes.fill_between(dic_aws_raw[var].rolling("1D", center=True).mean().index, dic_aws_raw[var]['lower_ci_95'].rolling("1D", center=True).mean(),
                        dic_aws_raw[var]['upper_ci_95'].rolling("1D", center=True).mean(), color="grey",
                alpha=0.5, label = "")
    else:
        axes.plot(dic_aws_raw[var]['median'].rolling("1D", center=True).mean(), color="black", label="AWS-forced Ens. Median", zorder=6)
        axes.plot(dic_cos_raw[var]['median'].rolling("1D", center=True).mean(), color="red", label="Posterior Ens. Median", zorder=5)
        axes.fill_between(dic_aws_raw[var]['median'].rolling("1D", center=True).mean().index, dic_cos_raw[var]['ci_lower'].rolling("1D", center=True).mean(),
                        dic_cos_raw[var]['ci_upper'].rolling("1D", center=True).mean(), color="brown",
                    alpha=0.5, label = "Posterior 95% CI")
        axes.fill_between(dic_aws_raw[var].rolling("1D", center=True).mean().index, dic_aws_raw[var]['lower_ci_95'].rolling("1D", center=True).mean(),
                        dic_aws_raw[var]['upper_ci_95'].rolling("1D", center=True).mean(), color="grey",
                alpha=0.5, label = "AWS-forced 95% CI")
    """
    axes.grid()
    
    
def plot_cdf(axes, var):
    xaws, yaws = calc_cdf(dic_aws_raw[var]['median'])
    axes.plot(xaws, yaws, color="black", lw=2)#, linestyle='none') #marker='.'
    xens, yens = calc_cdf(dic_cos_raw[var]['median'])
    axes.plot(xens, yens, color="red", lw=2)#, linestyle='none') #, marker='.'
    xcil, _ = calc_cdf(dic_cos_raw[var]['ci_lower'])
    xciu, _ = calc_cdf(dic_cos_raw[var]['ci_upper'])
    axes.fill_betweenx(yens, xcil, xciu, color='brown', alpha=0.6, label='95% CI')
    ycil, _ = calc_cdf(dic_aws_raw[var]['lower_ci_95'])
    yciu, _ = calc_cdf(dic_aws_raw[var]['upper_ci_95'])
    axes.fill_betweenx(yaws, ycil, yciu, color='grey', alpha=0.6, label='95% CI')


def plot_scatter(axes, var):
    up_aws, up_ens, up_z = sc_stats(dic_aws_raw[var]['median'], dic_cos_raw[var]['median'], drop_nan=True)
    print(up_aws)
    axes.scatter(up_aws.values, up_ens.values, c=up_z)
    # Set both axes to that range
    """
    axes.set_xlim(ylim_dic[var][0], ylim_dic[var][1])
    axes.set_ylim(ylim_dic[var][0], ylim_dic[var][1])
    
    x_min, x_max = axes.get_xlim()
    y_min, y_max = axes.get_ylim()

    # Find global min and max
    min_val = min(x_min, y_min)
    max_val = max(x_max, y_max)
    
    #axes.set_xlim(min_val, max_val)
    #axes.set_ylim(min_val, max_val)
    
    locator = MaxNLocator(nbins=3, prune=None)
    ticks = locator.tick_values(min_val, max_val)
    axes.set_xticks(ticks)
    axes.set_yticks(ticks)
    """
    lower, upper = ylim_dic[var]
    
    # Use MaxNLocator to get aesthetically pleasing ticks
    locator = MaxNLocator(nbins=3, prune=None)
    ticks = locator.tick_values(lower, upper)
    
    # Set ticks
    axes.set_xticks(ticks)
    axes.set_yticks(ticks)
    # Set same for both axes
    axes.set_xlim(lower, upper)
    axes.set_ylim(lower, upper)
    #
    """
    ci_upper_cos = dic_cos_ciupper[var].loc[dic_cos_ciupper[var].index.isin(up_aws)]
    ci_lower_cos = dic_cos_cilower[var].loc[dic_cos_cilower[var].index.isin(up_aws)]
    ci_upper_aws = dic_aws_ciupper[var].loc[dic_aws_ciupper[var].index.isin(up_aws)]
    ci_lower_aws = dic_aws_cilower[var].loc[dic_aws_cilower[var].index.isin(up_aws)]

    if var == "LWout":
        axes.errorbar(
            up_aws, up_ens, xerr=[up_aws - ci_upper_aws, ci_lower_aws - up_aws],
            yerr=[up_ens - ci_upper_cos, ci_lower_cos - up_ens],  # Switched because I had * -1
            fmt='none', ecolor='gray', alpha=0.3, capsize=2
        )
        
    else:
        axes.errorbar(
            up_aws, up_ens, xerr=[up_aws - ci_lower_aws, ci_upper_aws - up_aws],
            yerr=[up_ens - ci_lower_cos, ci_upper_aws - up_ens],  # Switched because I had * -1
            fmt='none', ecolor='gray', alpha=0.3, capsize=2
        )
    """
    # calculate metrics and add to subplot
    mbe = np.mean(up_ens - up_aws)
    rmse = np.sqrt(mean_squared_error(up_aws, up_ens))
    r2 = r2_score(up_aws, up_ens)
    textstr = f'MBE = {mbe:.3f}\nRMSE = {rmse:.3f}\nR² = {r2:.3f}'
    axes.text(0.05, 0.95, textstr, transform=axes.transAxes,
            fontsize=20, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

In [44]:
new_label_lookup = {'SWnet': r"$Q_{SWnet}$",
                    'LWnet': r"$Q_{LWnet}$",
                    'H': r"$Q_{H}$",
                    'LE': r"$Q_{E}$",
                    'B': r"$Q_{G}$",
                    'ME': r"$Q_{M}$",
                    'G': r"$Q_{SWin}$",
                    'LWin': r"$Q_{LWin}$",
                    'Q2': r'$Q_{2}$',
                    'RH2': 'RH2',
                    'PRES': 'Pres.',
                    'T2': 'T2',
                    'Td': r'$T_{d}$',
                    'U2': 'U2',
                    'REFREEZE': 'Refreezing',
                    'CONDENSATION': 'Condensation',
                    'EVAPORATION': 'Evaporation',
                    'SUBLIMATION': 'Sublimation',
                    'DEPOSITION': 'Deposition'}

In [45]:
def full_figure(vars, filename):
    var1, var2 = vars
    print(vars)
    fig = plt.figure(figsize=(26, 13)) # Adjusted figsize for the new layout
    
    # Create a 2x3 grid with custom width ratios
    # The first column will be twice as wide as the other two
    gs = gridspec.GridSpec(2, 3, figure=fig, width_ratios=[2, 1, 1])

    # Create an axes array manually to match the structure of plt.subplots
    ax = np.array([
        [fig.add_subplot(gs[0, 0]), fig.add_subplot(gs[0, 1]), fig.add_subplot(gs[0, 2])],
        [fig.add_subplot(gs[1, 0]), fig.add_subplot(gs[1, 1]), fig.add_subplot(gs[1, 2])]
    ])
    plot_ts(ax[0,0], var=var1)
    plot_ts(ax[1,0], var=var2)

    new_label = new_label_lookup[var]

    for xs in [ax[0,0],ax[1,0]]:
        if xs in [ax[0,0]]:
            try:
                xs.set_ylim(ylim_dic[var1][0], ylim_dic[var1][1])
            except:
                pass
            xs.set_ylabel(new_label_lookup[var1] + f" ({unit_dic[var1]})")
            #if var1 == "SWnet":
            #    xs.set_yticks(np.arange(0, 175+25,25))

        else:
            try:
                xs.set_ylim(ylim_dic[var2][0], ylim_dic[var2][1])
            except:
                pass
            xs.set_ylabel(new_label_lookup[var2] + f" ({unit_dic[var2]})")
            #if var2 == "LWnet":
            #    xs.set_yticks(np.arange(-85, 5+20,20))
        xs.set_xlabel("Date")
        xs.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(10, 1, 4, 7)))
        xs.xaxis.set_minor_locator(mdates.MonthLocator())
        xs.xaxis.set_major_formatter(
            mdates.ConciseDateFormatter(xs.xaxis.get_major_locator()))
            
    plot_cdf(ax[0,1], var=var1)
    plot_cdf(ax[1,1], var=var2)        
    for xs in [ax[0,1],ax[1,1]]:
        if xs in [ax[0,1]]:
            try:
                xs.set_xlim(ylim_dic[var1][0], ylim_dic[var1][1])
            except:
                pass
            xs.set_xlabel(new_label_lookup[var1] + f" ({unit_dic[var1]})")
            #if var == "SWnet":
            #    xs.xaxis.set_minor_locator(MultipleLocator(12.5))
            #    xs.set_xticks(np.arange(0, 175+25, 25))
        else:
            try:
                xs.set_xlim(ylim_dic[var2][0], ylim_dic[var2][1])
            except:
                pass
            xs.set_xlabel(new_label_lookup[var2] + f" ({unit_dic[var2]})")
            #if var == "LWnet":
            #    xs.xaxis.set_minor_locator(MultipleLocator(10))
            #    xs.set_xticks(np.arange(-85, 5+20, 20))       
        xs.set_ylabel("Probability (%)")
        xs.set_ylim(0, 1)
        xs.set_yticks(np.arange(0,1+0.1,0.1))
        xs.grid()
    #    elif var == "RH2":
    #        xs.xaxis.set_minor_locator(MultipleLocator(5))
    #        xs.set_xticks(np.arange(0, 100+10, 10))
    #    elif var == "LWout":
    #        xs.xaxis.set_minor_locator(MultipleLocator(10))
    #        xs.set_xticks(np.arange(150, 350+50, 50))    
    #    elif var == "LWin":
    #        xs.xaxis.set_minor_locator(MultipleLocator(10))
    #        xs.set_xticks(np.arange(0, 400+50, 50))        
    #    
        
    plot_scatter(ax[0,2], var=var1)
    plot_scatter(ax[1,2], var=var2)
    if var1 in ["T2","Td","U2","RH2","PRES","LWin","Q2","G","LWnet"]:
        lab_x_pre = "AWS"
        lab_y_pre = "COSMO"
    else:
        lab_y_pre = "Posterior Ens. Median"
        lab_x_pre = "AWS-assisted Ens. Median"
    ax[0,2].set_ylabel(f"{lab_y_pre} ({unit_dic[var1]})") #{new_label_lookup[var1]} 
    ax[0,2].set_xlabel(f"{lab_x_pre} ({unit_dic[var1]})") #{new_label_lookup[var1]} 
    #
    ax[1,2].set_ylabel(f"{lab_y_pre} ({unit_dic[var2]})") #{new_label_lookup[var2]} 
    ax[1,2].set_xlabel(f"{lab_x_pre} ({unit_dic[var2]})") #{new_label_lookup[var2]} 
    ax[0,2].axline((0, 0), slope=1, color='k', linestyle="-")
    ax[1,2].axline((0, 0), slope=1, color='k', linestyle="-")
    ax[0,2].grid()
    ax[1,2].grid()

    handles, labels = ax[0,0].get_legend_handles_labels()
    print(handles)
    print(labels)
    fig.tight_layout()
    fig.legend(handles, labels, loc='upper center', ncol=5, bbox_to_anchor=(0.5, -0.01))
    plt.savefig(filename, bbox_inches="tight")

In [ ]:
dic_aws_raw.keys()

In [ ]:
#PRES and U2
if 'win' in sys.platform:
    full_figure(vars=("PRES","U2"), filename="E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS16_AWSU_pointSEB_comp_PRES-U2.png")
else:
    full_figure(vars=("PRES","U2"), filename="/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS16_AWSU_pointSEB_comp_PRES-U2.png")

In [ ]:
if 'win' in sys.platform:
    full_figure(vars=("REFREEZE","SUBLIMATION"), filename="E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS22_AWSU_pointSEB_comp_REFREEZE-SUBLIMATION.png")
else:
    full_figure(vars=("REFREEZE","SUBLIMATION"), filename="/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS22_AWSU_pointSEB_comp_REFREEZE-SUBLIMATION.png")

In [ ]:
#G and LWin
#evaporation is roughly same
if 'win' in sys.platform:
    full_figure(vars=("DEPOSITION","CONDENSATION"), filename="E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS23_AWSU_pointSEB_comp_DEPOSITION-CONDENSATION.png")
else:
    full_figure(vars=("DEPOSITION","CONDENSATION"), filename="/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS23_AWSU_pointSEB_comp_DEPOSITION-CONDENSATION.png")

In [50]:
## Calculate specific humidity! -> Compare q, T2
# Parts of this inspired by COSIPY and ChatGPT
# MAgnus August Recehn blalba see COSIPY
def method_EW_Sonntag(T: float) -> float:
    # Create output array with the same shape as T
    Ew = np.empty_like(T, dtype=float)
    
    # Condition for water phase
    water_mask = T >= 273.16
    # Condition for ice phase
    ice_mask = T < 273.16
    
    # Calculate over water where condition is met
    Ew[water_mask] = 6.112 * np.exp((17.67 * (T[water_mask] - 273.16)) / (T[water_mask] - 29.66))
    
    # Calculate over ice where condition is met
    Ew[ice_mask] = 6.112 * np.exp((22.46 * (T[ice_mask] - 273.16)) / (T[ice_mask] - 0.55))
    
    return Ew

def calculate_specific_humidity(relative_humidity, temperature_celsius, pressure_hpa):
    temperature_kelvin = temperature_celsius + 273.15
    
    # saturation vapor pressure
    es = method_EW_Sonntag(temperature_kelvin)
    
    # actual vapor pressure
    e = (relative_humidity / 100.0) * es
    
    q = (0.622 * e) / (pressure_hpa - (1 - 0.622) * e)
    
    return q

def calculate_dew_point(relative_humidity, temperature_celsius):
    # 
    temperature_kelvin = temperature_celsius + 273.15
    es = method_EW_Sonntag(temperature_kelvin.values)
    
    #
    e = (relative_humidity.values / 100.0) * es
    
    # Create an output array for the results
    dew_point_celsius = np.empty_like(e, dtype=float)
    
    #
    e_safe = np.where(e > 0, e, 1e-10)
    val = np.log(e_safe / 6.112)

    #
    water_mask = e >= 6.112
    dew_point_celsius[water_mask] = (243.5 * val[water_mask]) / (17.67 - val[water_mask])

    #
    ice_mask = e < 6.112
    dew_point_celsius[ice_mask] = (272.61 * val[ice_mask]) / (22.46 - val[ice_mask])
    
    #
    return pd.Series(dew_point_celsius, index=temperature_celsius.index, name='dew_point')

aws_Td = calculate_dew_point(dic_aws_raw['RH2']['median'], dic_aws_raw['T2']['median'] - 273.15).rename('median')
cos_Td = calculate_dew_point(dic_cos_raw['RH2']['median'], dic_cos_raw['T2']['median'] - 273.15).rename('median')
# ci lower
aws_Td_cil = calculate_dew_point(dic_aws_raw['RH2']['lower_ci_95'], dic_aws_raw['T2']['lower_ci_95'] - 273.15).rename('lower_ci_95')
cos_Td_cil = calculate_dew_point(dic_cos_raw['RH2']['ci_lower'], dic_cos_raw['T2']['ci_lower'] - 273.15).rename('ci_lower')
# ci upper
aws_Td_ciu = calculate_dew_point(dic_aws_raw['RH2']['upper_ci_95'], dic_aws_raw['T2']['upper_ci_95'] - 273.15).rename('upper_ci_95')
cos_Td_ciu = calculate_dew_point(dic_cos_raw['RH2']['ci_upper'], dic_cos_raw['T2']['ci_upper'] - 273.15).rename('ci_upper')

# Calculate q
aws_q = calculate_specific_humidity(dic_aws_raw['RH2']['median'], dic_aws_raw['T2']['median']-273.15, dic_aws_raw['PRES']['median']) * 1000 #g/kg
cos_q = calculate_specific_humidity(dic_cos_raw['RH2']['median'], dic_cos_raw['T2']['median']-273.15, dic_cos_raw['PRES']['median']) * 1000 #g/kg
# ci lower
aws_q_cil = calculate_specific_humidity(dic_aws_raw['RH2']['lower_ci_95'], dic_aws_raw['T2']['lower_ci_95']-273.15, dic_aws_raw['PRES']['lower_ci_95']) * 1000 #g/kg
cos_q_cil = calculate_specific_humidity(dic_cos_raw['RH2']['ci_lower'], dic_cos_raw['T2']['ci_lower']-273.15, dic_cos_raw['PRES']['ci_lower']) * 1000 #g/kg
# ci upper
aws_q_ciu = calculate_specific_humidity(dic_aws_raw['RH2']['upper_ci_95'], dic_aws_raw['T2']['upper_ci_95']-273.15, dic_aws_raw['PRES']['upper_ci_95']) * 1000 #g/kg
cos_q_ciu = calculate_specific_humidity(dic_cos_raw['RH2']['ci_upper'], dic_cos_raw['T2']['ci_upper']-273.15, dic_cos_raw['PRES']['ci_upper']) * 1000 #g/kg
#q_g_per_kg = q_kg_per_kg * 1000 # Convert to g/kg for easier interpretation

df_q_aws = pd.concat([aws_q, aws_q_cil, aws_q_ciu], axis=1)
df_q_cos = pd.concat([cos_q, cos_q_cil, cos_q_ciu], axis=1)

dic_aws_raw['Q2'] = df_q_aws
dic_cos_raw['Q2'] = df_q_cos
#
df_Td_aws = pd.concat([aws_Td, aws_Td_cil, aws_Td_ciu], axis=1)
df_Td_cos = pd.concat([cos_Td, cos_Td_cil, cos_Td_ciu], axis=1)

dic_aws_raw['Td'] = df_Td_aws
dic_cos_raw['Td'] = df_Td_cos

In [ ]:
"""
if 'win' in sys.platform:
    full_figure(vars=("Td","T2"), filename="E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/AWSU_pointSEB_comp_Dew-T2.pdf")
else:
    full_figure(vars=("Td","T2"), filename="/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/AWSU_pointSEB_comp_Dew-T2.pdf")
"""

In [ ]:
"""
#G and LWin
if 'win' in sys.platform:
    full_figure(vars=("RH2","Q2"), filename="E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/AWSU_pointSEB_comp_RH2-Q.pdf")
else:
    full_figure(vars=("RH2","Q2"), filename="/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/AWSU_pointSEB_comp_RH2-Q.pdf")
"""

In [ ]:
if 'win' in sys.platform:
    full_figure(vars=("Q2","T2"), filename="E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS19_AWSU_pointSEB_comp_Q2-T2.png")
else:
    full_figure(vars=("Q2","T2"), filename="/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS19_AWSU_pointSEB_comp_Q2-T2.png")

In [ ]:
#G and LWin
if 'win' in sys.platform:
    full_figure(vars=("G","LWin"), filename="E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS17_AWSU_pointSEB_comp_SWin-LWin.png")
else:
    full_figure(vars=("G","LWin"), filename="/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS17_AWSU_pointSEB_comp_SWin-LWin.png")

In [ ]:
if 'win' in sys.platform:
    full_figure(vars=("SWnet","LWnet"), filename="E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS18_AWSU_pointSEB_comp_SWnet-LWnet.png")
else:
    full_figure(vars=("SWnet","LWnet"), filename="/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS18_AWSU_pointSEB_comp_SWnet-LWnet.png")


In [ ]:
if 'win' in sys.platform:
    full_figure(vars=("H","LE"), filename="E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS20_AWSU_pointSEB_comp_H-LE.png")
else:
    full_figure(vars=("H","LE"), filename="/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS20_AWSU_pointSEB_comp_H-LE.png")

In [ ]:
if 'win' in sys.platform:
    full_figure(vars=("B","ME"), filename="E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS21_AWSU_pointSEB_comp_B-ME.png")
else:
    full_figure(vars=("B","ME"), filename="/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigS21_AWSU_pointSEB_comp_B-ME.png")
